# Agent-Based Modeling of Marriages from Online Dating

#### Serena Chen and Apurva Raman

### Replicating the experiment in the [Strength of Absent Ties: Social Integration via Online Dating](https://arxiv.org/pdf/1709.10478.pdf.)

In [1]:
from __future__ import print_function, division

# %matplotlib inline
# %precision 3

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

# from matplotlib import rc
# rc('animation', html='html5')

In [2]:
def getGender(agents_per_race, race):
    women_per_race = agents_per_race//2
    women_matrix = np.ones((women_per_race, race))
    men_matrix= np.zeros((women_per_race, race))
    leftovers = np.random.randint(low=0, high=2, size=(agents_per_race-2*women_per_race,race))# high is exclusive, generates 0 and 1
    gender_matrix = np.concatenate((women_matrix, men_matrix, leftovers))
    
    # generate random matrix to shuffle columns of gender_matrix
    rand_matrix = np.random.random(gender_matrix.shape)
    
    row_indices = np.argsort(rand_matrix, axis=0)
    
    # Preserve order of columns
    col_indices = np.arange(gender_matrix.shape[1])[None, :]
   
    gender_matrix = gender_matrix[row_indices, col_indices]
    
    return np.transpose(gender_matrix)

In [3]:
agents_per_race = 100
races = 3
agents = agents_per_race*races
genders=getGender(agents_per_race,races)
print(genders)

[[ 1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  0.  1.  1.  1.
   0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  1.  0.  1.  1.  1.  0.  0.  0.
   1.  1.  0.  0.  1.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.  1.  1.  0.
   1.  0.  0.  1.  0.  0.  1.  1.  0.  1.  0.  0.  1.  0.  0.  1.  1.  1.
   1.  1.  1.  1.  1.  0.  1.  0.  0.  1.  0.  1.  1.  1.  1.  0.  1.  0.
   0.  0.  1.  0.  0.  1.  1.  1.  1.  0.]
 [ 0.  1.  0.  1.  1.  1.  1.  1.  1.  0.  0.  0.  1.  1.  1.  0.  0.  1.
   0.  1.  1.  1.  1.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  1.  1.  1.  0.  1.  0.  1.  0.  0.  0.  0.  1.  1.  0.  1.  0.  1.
   1.  1.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  1.  0.
   0.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.  1.  0.  1.  0.  1.
   1.  1.  0.  1.  0.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.
   1.  1.  1.  0.  0.  1.  1.  0.  1.  1.  1.  0.  0.  0.  0.  1.  1.  0.
   0.  0.  1.  1.  0.  1. 

In [4]:
def createPersonality(agents_per_race, race):
    social_beliefs = np.random.rand(race*agents_per_race)
    political_beliefs = np.random.rand(race*agents_per_race)
    return (social_beliefs, political_beliefs)
   

In [5]:
social_beliefs, political_beliefs = createPersonality(agents_per_race, races)
print ((social_beliefs,political_beliefs))

(array([  4.47199410e-01,   4.86089752e-01,   9.46765659e-01,
         3.11111117e-01,   5.69663435e-01,   2.31644114e-01,
         7.62020704e-01,   6.71460317e-01,   9.25500303e-01,
         5.80456820e-01,   3.11601886e-01,   7.70827411e-02,
         9.68713958e-01,   2.03756289e-01,   1.95267972e-01,
         5.19014154e-01,   9.37127021e-01,   8.66255258e-01,
         6.03817280e-01,   5.68391277e-01,   5.30307467e-01,
         5.76199803e-01,   4.96255426e-02,   2.23065079e-01,
         7.18549217e-01,   4.47157610e-01,   7.77295882e-01,
         3.58372032e-01,   2.01174858e-01,   2.98006197e-01,
         7.75504275e-01,   7.58944017e-01,   6.89870787e-01,
         4.48604929e-01,   8.47202607e-01,   4.14970107e-01,
         3.22364210e-02,   7.51335411e-01,   4.63035194e-01,
         2.09461146e-01,   2.41323783e-01,   7.79223190e-01,
         4.21163992e-01,   7.70832347e-01,   6.33897335e-01,
         7.84263170e-01,   7.01176345e-01,   7.11263486e-01,
         5.38755155e-01

In [6]:
def createAdj(agents_per_race, race, prob_inter=0.4, prob_intra=0.7):
    agents = agents_per_race*race
    adj= np.zeros((agents,agents))
    
    #Interracial edges
    max_inter = int(((race-1)*race/2)*agents_per_race**2)
    num_inter = int(max_inter*prob_inter)
    inter_edges = np.random.choice(max_inter, size=(1,num_inter), replace=False)
    inter_edge = 0
    for i in range(agents):
        prev_race = i//agents_per_race
        for j in range(prev_race*agents_per_race+agents_per_race, agents):
            if (i != j):
                if np.any(inter_edges == inter_edge):
                    adj[i,j] = 1
                inter_edge +=1    
                
    #Intraracial edges
    intra_edges = np.random.rand(agents,agents)
    for i in range(agents):
        current_race = i//agents_per_race
        for j in range (i+1, current_race*agents_per_race+agents_per_race):           
            if (intra_edges[i,j] < prob_intra):
                adj[i,j] = 1
                
    #Reflect triangle
    adj = np.triu(adj,0)+np.transpose(np.triu(adj))
    
    #Get all connections of at least distance 2 (at least one mutual friend)
    adj2 = np.matmul(adj,adj)
    
    #Directly connected or have at least one mutual friend
    adj3 = (adj+adj2>0).astype(int)
    np.fill_diagonal(adj3, 0)
    
    return adj,adj2,adj3

In [7]:
adj, adj2, adj3 = createAdj(agents_per_race, races)

In [8]:
# Distances

def getPersonalityDistances(agents, adjacency, genders, social_beliefs, political_beliefs):
    distance = np.zeros((agents,agents))
    gender_vector = genders.reshape((agents))
    
    for i in range(agents):
        for j in range(i, agents):
            # Same person
            if (i==j):
                distance[i, j] = np.nan
            elif (adjacency[i,j]==0):
                distance[i,j] = np.nan
            elif (gender_vector[i] == gender_vector[j]):
                distance[i, j] = np.nan
            else: 
                political_diff = political_beliefs[i] - political_beliefs[j]
                social_diff = social_beliefs[i] - social_beliefs[j]
                distance[i,j] = np.sqrt(political_diff**2 + social_diff**2)
    distance = np.triu(distance, 0)+np.transpose(np.triu(distance))
    return distance
        
        

In [9]:
distances = getPersonalityDistances(agents, adj3, genders, social_beliefs, political_beliefs)
print(distances)

[[        nan  0.40634081  0.66759324 ...,  0.40577266         nan
   0.71484319]
 [ 0.40634081         nan         nan ...,         nan  0.34923707
          nan]
 [ 0.66759324         nan         nan ...,         nan  0.11448949
          nan]
 ..., 
 [ 0.40577266         nan         nan ...,         nan  0.42938823
          nan]
 [        nan  0.34923707  0.11448949 ...,  0.42938823         nan
   0.69520654]
 [ 0.71484319         nan         nan ...,         nan  0.69520654
          nan]]


In [10]:
def createMarriages(agents, distances, genders):
    dist_copy = np.copy(distances)
    marriage = np.zeros(agents)
    marriage.fill(-1)
    
    women = np.sum(genders)
    men= agents-np.sum(genders)
    max_either_sex = int(max(women, men))
    best_match = np.nanargmin(dist_copy, axis=1)    
    
    for _ in range(max_either_sex):
        for i in range(agents):
            crush = best_match[i]
            if (i == best_match[crush]):
                marriage[i] = crush
        for j in range(agents):
            if (marriage[j]<0):
                if (marriage[best_match[j]] >= 0):
                    dist_copy[j, best_match[j]] = np.nan

        best_match = np.nanargmin(dist_copy, axis=1)   
    
    return (marriage.astype(int),distances)
                    

In [11]:
marriage, distances = createMarriages(agents, distances,genders)
print(marriage)
print(distances)

[279  86 197 120 138 190 208  45 218  84  87  52 260 217  28  21 205 273
 143  88 216  15  64  58  46 234  83 153  14  96 287 156  33  32 180 255
  93 196 282 112 237 116 139  82 169   7  24  68  51  78 281  48  11 155
  94 189 119 206  23 181 123 194 137 146  22 191 115 176  47 150 163 110
 222 102  91 283 236 104  49 292  85 270  43  26   9  80   1  10  19 157
 149  74 158  36  54  99  29 250 172  95 285 293  73 274  77 264 131 230
 221 248  71 148  39 177 280  66  41 152 262  56   3 130 200  60 179 231
 295 175 151 242 121 106 269 245 220 247 185  62   4  42 215 246  -1  18
 228 227  63 201 111  90  69 128 117  27 211  53  31  89  92 161 290 159
 226  70 193 195 219 187 209  44 252 173  98 171 192 127  67 113 294 124
  34  59 263 286 233 136 278 167 213  55   5  65 174 164  61 165  37   2
 297 268 122 147 296 267 223  16  57 289   6 168 272 154 299 188 225 140
  20  13   8 166 134 108  72 204 266 214 162 145 144 258 107 125 276 184
  25 241  76  40 284 240 239 235 129  -1 259 133 14

In [12]:
# Welfare

def averageDistances(distances, marriage):
    avg_dist = 0;
    for i in marriage:
        if (i >= 0):
            avg_dist += distances[i, marriage[i]]

    # Number of married people
    num_marriage = np.sum([marriage>=0])
    return(avg_dist/num_marriage)


In [13]:
avg_dist = averageDistances(distances, marriage)
print(avg_dist)

0.0867032437729


In [14]:
def numIntraracial(marriage, agents_per_race, races):
    agents = agents_per_race*races
    num_intra = 0
    for i in range(agents):
        if (marriage[i] >=0):
            curr_race = i//agents_per_race
            race_end = (1+curr_race) * agents_per_race
            race_start = curr_race*agents_per_race
            if ((marriage[i] < race_end) and (marriage[i]>=race_start)):
                num_intra += 1
    return num_intra/2

In [15]:
num_intra = numIntraracial(marriage, agents_per_race, races)
print(num_intra)

59.0


In [16]:
def numInterracial(num_intra, marriage):
    # Number of marriages
    num_marriage = np.sum([marriage>=0])/2
    return(num_marriage - num_intra) 

In [17]:
num_inter = numInterracial(num_intra, marriage)
print(num_inter)

90.0


In [18]:
def welfareRatios(num_intra, num_inter, marriage, races, agents_per_race, avg_dist):
    num_marriage = np.sum([marriage>=0])/2
    
    #ratio of people in interracial marriages to some race thing
    
    #TODO what is this
    r1 = (num_inter/num_marriage)/((races-1)/races)
        
    #ratio of people in marriages to total people
    r2 = num_marriage*2/(agents_per_race*races)
    
    #Normalized compatibility
    r3 = (np.sqrt(2)-avg_dist)/np.sqrt(2)
    
    return(r1,r2,r3)

In [19]:
r1,r2,r3 = welfareRatios(num_intra, num_inter, marriage, races, agents_per_race, avg_dist)
print(r1,r2,r3)

0.906040268456 0.993333333333 0.938691548377
